In [17]:
"""
Calculate fire severity statistics within AFD observations from MODIS and VIIRS.
Severity index: Composite Burn Severity (CBI), calculated in GEE via Parks (2018)

Author: maxwell.cook@colorado.edu
"""

import os, sys
import ee

# Custom functions
sys.path.append(os.path.join(os.getcwd(),'code/'))
from __functions import *

ee.Authenticate()
ee.Initialize(project='jfsp-aspen')

print("Success !")

Success !


In [5]:
# Load the gridded FRP data
grid = ee.FeatureCollection('projects/jfsp-aspen/assets/AFD/viirs_snpp_jpss1_afd_gridstats')
print(f"Number of grid cells: {grid.size().getInfo()}")
grid = grid.select(['grid_index','Fire_ID'])
# Align the grid geometries with the GridMET projection
# Define the DEM projection explicitly
dem_proj = ee.ImageCollection('USGS/3DEP/10m_collection').first().projection()
# Reproject geometries properly
grid = grid.map(lambda f: f.setGeometry(
    f.geometry().transform(dem_proj, 30)  # 30 m tolerance for reprojection
))
print(grid.first().propertyNames().getInfo())

Number of grid cells: 56974
['system:index', 'grid_index', 'Fire_ID']


In [46]:
# Load topographic collections
from tagee import terrainAnalysis

# set up a smoothed DEM
gaussianFilter = ee.Kernel.gaussian(
  radius=3, sigma=2, units='pixels', normalize=True
)
srtmSmooth = ee.Image("USGS/SRTMGL1_003").convolve(gaussianFilter).resample("bilinear")

# Calculate terrain metrics over a given geometry.
terrainMetrics = terrainAnalysis(srtmSmooth)
terrainMetrics = terrainMetrics.select(['Elevation', 'Slope', 'Northness', 'Eastness'], ['elevation','slope','northness','eastness'])

# Continuous Heat-Insolation Load Index (CHILI)
chili = ee.Image("CSP/ERGo/1_0/US/CHILI").rename('chili')

# --- Combine with slope and elevation ---
terrainMetrics = terrainMetrics.addBands(chili)
print(terrainMetrics.bandNames().getInfo())

['elevation', 'slope', 'northness', 'eastness', 'chili']


In [47]:
# Run the reduction on the grid
dem_stats = terrainMetrics.reduceRegions(
    collection=grid,
    reducer=ee.Reducer.mean(),
    scale=30
)

# run TPI separately
# Topographic Position Index (TPI) (USGS NED 10m-based, 270m res)
tpi = ee.Image("CSP/ERGo/1_0/US/mTPI").rename('tpi')
tpi_stats = tpi.reduceRegions(
    collection=grid,
    reducer=ee.Reducer.mode().setOutputs(['tpi']),
    scale=270, # resolution of the TPI
    tileScale=2
)

# merge these
# Define a join filter
join_filter = ee.Filter.equals(
    leftField='grid_index',
    rightField='grid_index'
)

# Apply an inner join (keeps matching features)
joined = ee.Join.inner().apply(dem_stats, tpi_stats, join_filter)

# Merge properties into single feature
def merge_feats(f):
    left = ee.Feature(f.get('primary'))
    right = ee.Feature(f.get('secondary'))
    return left.copyProperties(right)

# Create final merged collection
topo_stats = ee.FeatureCollection(joined.map(merge_feats))

# Pull a few features into Python to inspect
sample = topo_stats.limit(10).getInfo()
props = [f['properties'] for f in sample['features']]
df = pd.DataFrame(props)
df

,Fire_ID,chili,eastness,elevation,grid_index,northness,slope,tpi
0,71,184.530786,0.013091,2621.674122,1766660,0.906543,9.964018,2
1,71,181.574906,0.327737,2586.930004,1766661,0.761077,10.120274,-8
2,71,197.110966,-0.380865,2557.561706,1766662,0.400299,12.572239,-11
3,71,205.781335,-0.688227,2539.404715,1766663,0.300543,10.692603,-41
4,71,195.675797,0.754680,2619.043826,1768909,0.100018,7.323137,5
5,71,190.175313,-0.007775,2613.639388,1768910,0.470546,9.802152,-6
6,71,198.659918,-0.478300,2605.966335,1768913,0.116766,18.635969,-18
7,71,215.767483,-0.209094,2602.310283,1771159,-0.422815,13.771395,-12
8,71,219.775477,-0.670701,2639.941341,1771160,-0.616474,11.967990,15
9,71,206.061009,-0.496901,2656.379124,1771163,-0.005822,17.295700,2


In [48]:
# Remove geometry
def remove_geometry(ftr):
    return ftr.setGeometry(None)
topo_stats = topo_stats.map(remove_geometry)

# export to drive
export_task = ee.batch.Export.table.toDrive(
    collection=topo_stats,
    description='gridstats_topo_v2',
    fileNamePrefix='gridstats_topo_v2',
    fileFormat='CSV', 
    folder='TOPO_V2'
)

export_task.start() # Start the export task
print("Export to Earth Engine Asset started!")
# Monitor the task until it's finished
monitor_export(export_task, 60)

Export to Earth Engine Asset started!
Waiting for export to finish..
	Patience young padawan.
Waiting for export to finish..
	Patience young padawan.
Waiting for export to finish..
	Patience young padawan.
Waiting for export to finish..
	Patience young padawan.
Waiting for export to finish..
	Patience young padawan.
Waiting for export to finish..
	Patience young padawan.
Waiting for export to finish..
	Patience young padawan.
Waiting for export to finish..
	Patience young padawan.
Waiting for export to finish..
	Patience young padawan.
Waiting for export to finish..
	Patience young padawan.
Waiting for export to finish..
	Patience young padawan.
Waiting for export to finish..
	Patience young padawan.
Waiting for export to finish..
	Patience young padawan.
Waiting for export to finish..
	Patience young padawan.
Waiting for export to finish..
	Patience young padawan.
Waiting for export to finish..
	Patience young padawan.
Export completed successfully !!!!
